# Advent of Code 2016
[see here](http://adventofcode.com/2016)

## Preparation
This is inspired by [Peter Norvigs solution](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [65]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, reduce
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    
    filename = 'input/input{}.txt'.format(day)
    try:
        with open(filename, 'r') as f:
            text = f.read()
        return text
    except FileNotFoundError:
        url = 'http://adventofcode.com/2016/day/{}/input'.format(day)
        print('input file not found. opening browser...')
        import webbrowser
        webbrowser.open(url)

cat = ''.join

def prod(it):
    return reduce(lambda x,y: x*y, it)

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (X+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

## Day 1

In [16]:
Point = complex             
N, S, E, W = 1j, -1j, 1, -1 # Unit vectors for headings

def distance(point): 
    "City block distance between point and the origin."
    return abs(point.real) + abs(point.imag)

def how_far(moves):
    "After following moves, how far away from the origin do we end up?"
    loc, heading = 0, N # Begin at origin, heading North
    for (turn, dist) in parse(moves):
        heading *= turn
        loc += heading * dist
    return distance(loc)

def parse(text):
    "Return a list of (turn, distance) pairs from text of form 'R2, L42, ...'"
    turns = dict(L=N, R=S)
    return [(turns[RL], int(d))
           for (RL, d) in re.findall(r'(R|L)(\d+)', text)]

assert distance(Point(3, 4)) == 7 # City block distance; Euclidean distance would be 5
assert parse('R2, L42') == [(S, 2), (N, 42)]
assert how_far("R2, L3") == 5
assert how_far("R2, R2, R2") == 2
assert how_far("R5, L5, R5, R3") == 12
how_far(Input(1))

239.0

In [3]:
"""Part 2. first place that is visited twice."""

Point = complex             
N, S, E, W = 1j, -1j, 1, -1 # Unit vectors for headings

def distance(point): 
    "City block distance between point and the origin."
    return abs(point.real) + abs(point.imag)

def how_far(moves):
    "After following moves, how far away from the origin do we end up?"
    visited = set()
    loc, heading = 0, N # Begin at origin, heading North
    for (turn, dist) in parse(moves):
        heading *= turn
        for _ in range(dist):
            loc += heading
            if loc in visited:
                return distance(loc)
            else:
                visited.add(loc)
    return distance(loc)

def parse(text):
    "Return a list of (turn, distance) pairs from text of form 'R2, L42, ...'"
    turns = dict(L=N, R=S)
    return [(turns[RL], int(d))
           for (RL, d) in re.findall(r'(R|L)(\d+)', text)]

how_far(Input(1))

141.0

## Day 2

In [29]:
Keypad = str.split

keypad = Keypad("""
.....
.123.
.456.
.789.
.....
""")

assert keypad[2][2] == '5'

off = '.'

def decode(instructions, x=2, y=2):
    """Follow instructions, keeping track of x, y position, and
    yielding the key at the end of each line of instructions."""
    for line in instructions:
        for C in line:
            x, y = move(C, x, y)
        yield keypad[y][x]

def move(C, x, y):
    "Make the move corresponding to this character (L/R/U/D)"
    if   C == 'L' and keypad[y][x-1] is not off: x -= 1
    elif C == 'R' and keypad[y][x+1] is not off: x += 1
    elif C == 'U' and keypad[y-1][x] is not off: y -= 1
    elif C == 'D' and keypad[y+1][x] is not off: y += 1
    return x, y

assert move('U', 2, 2) == (2, 1)
assert move('U', 2, 1) == (2, 1)
assert cat(decode("ULL RRDDD LURDL UUUUD".split())) == '1985'

cat(decode(Input(2).strip().split('\n')))

'56983'

In [31]:
keypad = Keypad("""
.......
...1...
..234..
.56789.
..ABC..
...D...
.......
""")

assert keypad[3][1] == '5'

cat(decode(Input(2).strip().split('\n'), x=1, y=3))

'8B8B1'

## Day 3

In [32]:
def is_triangle(sides):
    a,b,c = sorted(sides)
    return a+b>c and a+c>b and b+c>a

def parse(text):
    triangle_list = []
    for line in text.split('\n'):
        sides = tuple(int(word) for word in re.findall(r'\S+',line))
        if sides:
            triangle_list.append(sides)
    return triangle_list

triangles = parse(Input(3))
print('part 1: ' + str(len([tri for tri in triangles if is_triangle(tri)])))
from itertools import chain, zip_longest
triangles2 = chain(*zip(*triangles))
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)
triangles2 = grouper(triangles2,3)
print('part 2: ' + str(len([tri for tri in triangles2 if is_triangle(tri)])))

part 1: 917
part 2: 1649


## Day 4 

In [33]:
from collections import Counter
rooms = Input(4).strip().split('\n')
pattern = re.compile(r'(?P<name>[-a-z]+)-(?P<sector>\d+)\[(?P<check>[a-z]+)\]')

def parse_room(room):
    return pattern.match(room).groupdict()

def check_room(room):
    c = Counter(room['name'].replace('-',''))
    check = cat(sorted(c, key=lambda x: (-c[x],x))[:5])
    return check == room['check']
    
rooms = [parse_room(room) for room in rooms]
rooms = [room for room in rooms if check_room(room)]
sum(int(room['sector']) for room in rooms)

173787

In [34]:
def alpha_rot(n):
    return cat(chr(ord('a')+(i+n)%26) for i in range(26))
def decrypt_name(n):
    t = str.maketrans(alpha_rot(0)+'-', alpha_rot(n)+' ')
    return lambda s: s.translate(t)
assert decrypt_name(343)('qzmt-zixmtkozy-ivhz') == 'very encrypted name'
for room in rooms:
    sector = int(room['sector'])
    dec = decrypt_name(sector)
    dec_name = dec(room['name'])
    if 'pole' in dec_name:
        print(room['sector'] + ' ' + dec_name)

548 northpole object storage


## Day 5

In [170]:
door_id = 'abbhdwsy'
import hashlib
def door_md5(door_id, i):
    m = hashlib.md5()
    m.update(bytes(door_id + str(i),'utf-8'))
    return m.hexdigest()

assert door_md5('abc',3231929)[:6] == '000001'

def gen_pwd(door_id):
    pwd = ''
    i = 0
    while len(pwd) < 8:
        h = door_md5(door_id, i)
        if h.startswith('00000'):
            pwd += h[5]
        i+=1
    return pwd

#assert gen_pwd('abc') == '18f47a30'
gen_pwd(door_id)

'801b56a7'

In [185]:
def gen_pwd2(door_id):
    pwd = '_'*8
    i = 0
    print(pwd)
    while '_' in pwd:
        h = door_md5(door_id, i)
        pos = int(h[5],base=16)
        if h.startswith('00000') and pos < 8 and pwd[pos] == '_':
            pwd = pwd[:pos]+ h[6] + pwd[pos+1:]
            print(pwd)
        i+=1
    return pwd
gen_pwd2(door_id)

________
4_______
42______
42___1__
42___19_
42___197
424__197
424_0197
424a0197


'424a0197'

## Day 6

In [197]:
msgs = Input(6).split()
cols = list(zip(*msgs))
from collections import Counter
message = cat(Counter(col).most_common(1)[0][0] for col in cols)
print(message)
message2 = cat(Counter(col).most_common()[-1][0] for col in cols)
print(message2)

liwvqppc
caqfbzlh


## Day 7

In [40]:
ips = Input(7).split()
abba = re.compile(r'(\w)(?!\1)(\w)\2\1')
split = re.compile(r'(\w+)+')

def check_TLS(ip):
    parts = split.findall(ip)
    good_abba = False
    bad_abba = False
    for i, part in enumerate(parts):
        if abba.search(part):
            if i%2: #0-> outside of []. 1-> inside of []
                bad_abba = True
            else:
                good_abba = True
    return good_abba and not bad_abba

assert check_TLS('abba[mnop]qrst') == True
assert check_TLS('abcd[bddb]xyyx') == False
assert check_TLS('aaaa[qwer]tyui') == False
assert check_TLS('ioxxoj[asdfgh]zxcvbn') == True

def check_SSL(ip):
    pass

assert check_SSL('aba[bab]xyz')
assert check_SSL('xyx[xyx]xyx') == False
assert check_SSL('aaa[kek]eke')
assert check_SSL('zazbz[bzb]cdb')

ips = [ip for ip in ips if check_TLS(ip)]
len(ips)


110

## Day 8
Format for instructions:
```
rect 2x1
rotate row y=0 by 5
rotate column x=0 by 1
```

In [41]:
instructions = Input(8).splitlines()
rect = re.compile(r'rect (?P<width>\d+)x(?P<height>\d+)')
rotr = re.compile(r'rotate row y=(?P<row>\d+) by (?P<shift>\d+)')
rotc = re.compile(r'rotate column x=(?P<col>\d+) by (?P<shift>\d+)')

def get_param(pattern, text):
    match = pattern.findall(text)
    if not match: return
    return tuple(int(x) for x in match[0])

FILLCHAR = '@'
WIDTH, HEIGHT = 50,6
disp = [[' ' for _ in range(WIDTH)] for _ in range(HEIGHT)]
for line in instructions:
    param = get_param(rect,line)
    if param:
        width, height = param
        for i, j in product(range(width), range(height)):
            disp[j][i] = FILLCHAR
        continue
    param = get_param(rotr, line)
    if param:
        row, shift = param
        disp[row] = disp[row][-shift:] + disp[row][:-shift]
        continue
    param = get_param(rotc, line)
    if param:
        disp = list(zip(*disp))
        row, shift = param
        disp[row] = disp[row][-shift:] + disp[row][:-shift]
        disp = list(list(row) for row in zip(*disp))
        continue
dispstr = '\n'.join(cat(line) for line in disp)
print('#Pixels lit: '+ str(dispstr.count(FILLCHAR)))
print('Password:')
print(dispstr)

#Pixels lit: 121
Password:
@@@  @  @ @@@  @  @  @@  @@@@  @@  @@@@  @@@ @    
@  @ @  @ @  @ @  @ @  @ @    @  @ @      @  @    
@  @ @  @ @  @ @  @ @    @@@  @  @ @@@    @  @    
@@@  @  @ @@@  @  @ @    @    @  @ @      @  @    
@ @  @  @ @ @  @  @ @  @ @    @  @ @      @  @    
@  @  @@  @  @  @@   @@  @@@@  @@  @@@@  @@@ @@@@ 


## Day 9

In [31]:
ctext = Input(9).strip()
mark = re.compile(r'\((?P<len>\d+)x(?P<rep>\d+)\)')
def decompress(ctext):
    dtext = ''
    pos = 0
    while pos < len(ctext):
        m = mark.search(ctext, pos)
        if not m:
            dtext += ctext[pos:]
            pos = len(ctext)
            break
        start = m.start()
        end = m.end()
        length, repeat = int(m.groupdict()['len']), int(m.groupdict()['rep'])
        dtext += ctext[pos:start]
        dtext += ctext[end:end+length] * repeat
        pos = end+length
    return dtext

assert decompress('ADVENT') == 'ADVENT'
assert decompress('A(1x5)BC') == 'ABBBBBC'
assert decompress('(3x3)XYZ') == 'XYZXYZXYZ'
assert decompress('A(2x2)BCD(2x2)EFG') == 'ABCBCDEFEFG'
assert decompress('(6x1)(1x3)A') == '(1x3)A'
assert decompress('X(8x2)(3x3)ABCY') == 'X(3x3)ABC(3x3)ABCY'


dtext = decompress(ctext)
print(dtext[:200] + ' ...')
print('Total Length: ' + str(len(dtext)))

(22x7)(4x15)XOPG(7x9)JDPAKGM(8x8)ALGCJRZQ(38x1)(4x10)VNSW(12x10)BZPAZABYKIDJ(3x14)IHF(40x15)(34x8)UGTIHCTVONZPPIWUAEGHGFJUNTIMIELOLW(6x1)XLMMKD(22x7)(4x15)XOPG(7x9)JDPAKGM(8x8)ALGCJRZQ(38x1)(4x10)VNSW ...
Total Length: 107035


In [37]:
def decompress2len(ctext):
    dtext = 0 #is now length
    pos = 0
    while pos < len(ctext):
        m = mark.search(ctext, pos)
        if not m:
            dtext += len(ctext) - pos
            pos = len(ctext)
            break
        start = m.start()
        end = m.end()
        length, repeat = int(m.groupdict()['len']), int(m.groupdict()['rep'])
        dtext += start-pos
        dtext += decompress2len(ctext[end:end+length]) * repeat
        pos = end+length
    return dtext


assert decompress2len('(27x12)(20x12)(13x14)(7x10)(1x12)A') == 241920
assert decompress2len('(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN') == 445

dtext = decompress2len(ctext)
print('Total Length: ' + str(dtext))

Total Length: 11451628995


## Day 10

In [69]:
instr = Input(10).strip()
TARGET = set((17,61))
#print(log[:497] + '\n...')
pat_rec = re.compile(r'value (\d+) goes to (\w+ \d+)')
pat_giv = re.compile(r'(\w+ \d+) gives low to (\w+ \d+) and high to (\w+ \d+)')

def give(stash, recipient, value, target=None):
    stash[recipient].add(value)
    if len(stash[recipient]) == 2:
        s = stash[recipient]
        if s == target: print('{} has {}'.format(recipient,s))
        stash[recipient] = set()
        give(stash, flow[recipient][0], min(s),target)
        give(stash, flow[recipient][1], max(s),target)

flow = {giver: (low, high) for giver, low, high in pat_giv.findall(instr)}
stash = defaultdict(set)
for val, dest in pat_rec.findall(instr):
    give(stash, dest, int(val),TARGET)

outputs = ['output '+str(i) for i in range(3)]
print('{} = {}'.format(' * '.join(outputs), prod(stash[o].pop() for o in outputs)))

bot 27 has {17, 61}
output 0 * output 1 * output 2 = 13727
